In [3]:
import pandas as pd 
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import os
from db import query_to_df


### Purpose
In this notebook we look to analyse a teams previous performance in the league and how this may affect their performance in the next season. We will look at the following:
- What stats are most impactful in determinig a teams performance?
- How does a teams performance in the previous season affect their performance in the next season?
- How does a teams performance in the previous season affect their performance in the next season when we take into account the stats that are most impactful?
- Do teams with certain types of players match up against each other well
- Can we apply a weighting to a teams players to determine the strength of a team when they come up against another team?

In [14]:
# Get all the teams from the teams table
teams = query_to_df("SELECT * FROM teams")
# Only keep name and provider_id columns

teams.head()

,id,provider_id,name,abbreviation,nickname
0,1,CD_T10,Adelaide Crows,ADEL,Crows
1,2,CD_T20,Brisbane Lions,BL,Lions
2,3,CD_T40,Collingwood,COLL,Magpies
3,4,CD_T1000,Gold Coast Suns,GCFC,Suns
4,5,CD_T30,Carlton,CARL,Blues


In [17]:
# Get seasons from the seasons table
seasons = query_to_df("SELECT * FROM afl_seasons")

seasons.head()
# ID is used for all basic stats
# Provider_id is used for AFL Stats pro

,id,provider_id,year,name
0,53,CD_S2023014,2023,2023 Toyota AFL Premiership
1,43,CD_S2022014,2022,2022 Toyota AFL Premiership
2,62,CD_S2024014,2024,2024 Toyota AFL Premiership
3,34,CD_S2021014,2021,2021 Toyota AFL Premiership
4,20,CD_S2020014,2020,2020 Toyota AFL Premiership


In [19]:
# Get AFL fixture from the fixtures table
# Fixtures table also includes the results of each game if they have been played
fixture_df = query_to_df("SELECT * FROM afl_fixture WHERE comp_season_id = 52")
fixture_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   id                      216 non-null    int64              
 1   round_id                216 non-null    int64              
 2   comp_season_id          216 non-null    int64              
 3   round_number            216 non-null    int64              
 4   venue                   216 non-null    object             
 5   venue_abbreviation      216 non-null    object             
 6   venue_state             216 non-null    object             
 7   utc_start_time          216 non-null    datetime64[ns, UTC]
 8   status                  216 non-null    object             
 9   home_team_id            216 non-null    int64              
 10  home_team_name          216 non-null    object             
 11  home_team_abbreviation  216 non-null    objec

In [21]:
# Isolate a single team to test with
team = teams[teams['name'] == 'Brisbane Lions']
team_id = team['id'].values[0]

# Filter fixture to only teams games
team_fixture_df = fixture_df[(fixture_df['home_team_id'] == team_id) | (fixture_df['away_team_id'] == team_id)]



,id,round_id,comp_season_id,round_number,venue,venue_abbreviation,venue_state,utc_start_time,status,home_team_id,...,home_team_goals,home_team_behinds,home_team_total,away_team_id,away_team_name,away_team_abbreviation,away_team_nickname,away_team_goals,away_team_behinds,away_team_total
3,4788,758,52,1,Adelaide Oval,AO,SA,2023-03-18 05:35:00+00:00,CONCLUDED,7,...,18,18,0,2,Brisbane Lions,BL,Lions,11,6,0
10,4794,759,52,2,Gabba,G,QLD,2023-03-24 08:55:00+00:00,CONCLUDED,2,...,14,9,0,17,Melbourne,MELB,Demons,13,4,0
18,4803,760,52,3,Marvel Stadium,MRVL,VIC,2023-03-30 08:20:00+00:00,CONCLUDED,8,...,10,7,0,2,Brisbane Lions,BL,Lions,7,11,0
27,4812,761,52,4,Gabba,G,QLD,2023-04-06 09:35:00+00:00,CONCLUDED,2,...,18,8,0,3,Collingwood,COLL,Magpies,11,17,0
39,4823,762,52,5,Adelaide Hills,AHL,SA,2023-04-15 03:10:00+00:00,CONCLUDED,2,...,22,20,0,6,North Melbourne,NMFC,Kangaroos,12,5,0
47,4832,763,52,6,Manuka Oval,MO,ACT,2023-04-22 06:35:00+00:00,CONCLUDED,15,...,13,9,0,2,Brisbane Lions,BL,Lions,16,12,0
55,4839,764,52,7,Gabba,G,QLD,2023-04-29 03:45:00+00:00,CONCLUDED,2,...,17,13,0,14,Fremantle,FRE,Dockers,10,7,0
63,4846,765,52,8,Marvel Stadium,MRVL,VIC,2023-05-05 09:50:00+00:00,CONCLUDED,5,...,11,8,0,2,Brisbane Lions,BL,Lions,15,10,0
77,4863,766,52,9,Gabba,G,QLD,2023-05-13 09:25:00+00:00,CONCLUDED,2,...,12,15,0,12,Essendon,ESS,Bombers,6,9,0
85,4870,767,52,10,Gabba,G,QLD,2023-05-20 09:30:00+00:00,CONCLUDED,2,...,16,11,0,4,Gold Coast Suns,GCFC,Suns,9,10,0
